Use chap4.py to get the 3-currency data used in Chapter 4 of QRPM from the beginning of 1971 to the end of 2021. Take the year 2021 out of this data (the holdout set; the data before that is the learning set. Compute the learning set's equality-constrained efficient frontier as in the formulas for its means and standard deviation.

In [1]:
#chap4.py
import pandas as pd
import qrpm_funcs as qf
#Get 3 currencies until the end of
#previous year. Form sample covariance matrix
#and do simple efficient frontier calculations

lastday=qf.LastYearEnd()
#Swiss franc, pound sterling, Japanese Yen
seriesnames=['DEXSZUS','DEXUSUK','DEXJPUS']
cdates,ratematrix=qf.GetFREDMatrix(seriesnames,enddate=lastday)
multipliers=[-1,1,-1]

lgdates,difflgs=qf.levels_to_log_returns(cdates,ratematrix,multipliers)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


### Use the formulas for the efficient frontier

The formulas for the equality-constrained efficient frontier are:

$$
\mu = \frac{\sum_{i} w_i \mu_i}{\sum_{i} w_i}
$$
Where:
- $ \mu $ = Portfolio return
- $ w_i $ = Weight of asset i in the portfolio (equal weights for all assets in this case)
- $ \mu_i $ = Return of asset i

$$
\sigma = \sqrt{w^T \Sigma w}
$$
Where:
- $ \sigma $ = Portfolio standard deviation
- $ w $ = Vector of portfolio weights
- $ \Sigma $ = Covariance matrix



In [2]:
import pandas as pd
import qrpm_funcs as qf
import numpy as np

# Convert the list of dates to a pandas datetime format
lgdates_dt = pd.to_datetime(lgdates)

# Convert the log returns to a DataFrame for easier slicing
df = pd.DataFrame(difflgs, index=lgdates_dt)

# Extract the data for the year 2021
holdout_data = df[df.index.year == 2021]
learning_data = df[df.index.year < 2021]

# Calculate mean returns and the sample covariance matrix for the data up to 2020
mean_returns = learning_data.mean()
cov_matrix = learning_data.cov()

# Calculate the equality-constrained efficient frontier
n_assets = len(seriesnames)
equal_weights = np.repeat(1/n_assets, n_assets)

portfolio_return = np.dot(equal_weights, mean_returns)
portfolio_std_dev = np.sqrt(np.dot(equal_weights.T, np.dot(cov_matrix, equal_weights)))

print("Equality-Constrained Portfolio Return:", portfolio_return)
print("Equality-Constrained Portfolio Standard Deviation:", portfolio_std_dev)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Equality-Constrained Portfolio Return: 5.929673476567304e-05
Equality-Constrained Portfolio Standard Deviation: 0.005182871343279609


In [3]:
import numpy as np

def compute_optimal_weights(cov_matrix, mean_returns, lambda_value):
    """Compute the optimal portfolio weights."""
    inv_cov = np.linalg.inv(cov_matrix)
    numerator = np.dot(inv_cov, mean_returns)
    denominator = lambda_value + np.dot(np.ones_like(mean_returns), numerator)
    return numerator / denominator

lambdas = [0.01 * i for i in range(11)]
learning_variances = []
holdout_variances = []

# Compute portfolio weights for each lambda and calculate variance in both sets
for lambda_value in lambdas:
    weights = compute_optimal_weights(cov_matrix, mean_returns, lambda_value)
    
    # Portfolio variance in learning set
    learning_variance = np.dot(weights.T, np.dot(cov_matrix, weights))
    learning_variances.append(learning_variance)
    
    # Portfolio variance in holdout set
    holdout_cov_matrix = holdout_data.cov()
    holdout_variance = np.dot(weights.T, np.dot(holdout_cov_matrix, weights))
    holdout_variances.append(holdout_variance)

# Check order of variances
learning_order = np.argsort(learning_variances)
holdout_order = np.argsort(holdout_variances)

print("Learning Set Variances Order:", learning_order)
print("Holdout Set Variances Order:", holdout_order)

# Check if the order of variances is the same in both sets
if np.array_equal(learning_order, holdout_order):
    print("The order of variances is the SAME in both sets.")
else:
    print("The order of variances is DIFFERENT in both sets.")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
